<a href="https://colab.research.google.com/github/OVP2023/NLP/blob/main/dict_lesson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
# Задание:  Из текста - отдельных документов выделить адреса

In [ ]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/Datasets/NLTK_neto/base_address_ner_dataset.csv')
data.head(5)

,text_part
0,"771001001 кпп по октмо гбу ""жилищник района м..."
1,"5147746350230, окато 45283559000 октмо 453670..."
2,"он. статья 15 адреса, реквизиты и подписи стор..."
3,сударственное бюджетное учреждение гор. москвы...
4,остью «дорстройгрупп» (ооо «дорстройгрупп» юри...


In [4]:
data.text_part[0]

' 771001001 кпп по октмо гбу "жилищник района митино" государственные бюджетные учреждения субъектов российской федерации 7733902394 кпп 773301001 по окфс место нахождения (адрес) S125430, город москва, пятницкое шоссе, дом 23E реквизиты сведений об объекте контроля 45000000 инн по окопф форма собственности № п/п дата протокола по октмо 75203 29368873 45367000000 реквизиты документа, содержащего информацию для осуществления'

In [5]:
data.text_part = data.text_part.str.replace('S', ' S ').str.replace('E', ' E ').str.replace('\s\s+', ' ')
data.text_part

,text_part
0,"771001001 кпп по октмо гбу ""жилищник района м..."
1,"5147746350230, окато 45283559000 октмо 453670..."
2,"он. статья 15 адреса, реквизиты и подписи стор..."
3,сударственное бюджетное учреждение гор. москвы...
4,остью «дорстройгрупп» (ооо «дорстройгрупп» юри...
...,...
741,тракту №обю/19-1 локальная смета № (локальный ...
742,ткая марка по плотности 70 длина 1000; 2000 мм...
743,"т 19.06.2017 № 508671, выданной публичным акци..."
744,и сжатии 300 или 400 400 вид цемента портландц...


In [6]:
data.text_part[0]

' 771001001 кпп по октмо гбу "жилищник района митино" государственные бюджетные учреждения субъектов российской федерации 7733902394 кпп 773301001 по окфс место нахождения (адрес)  S 125430, город москва, пятницкое шоссе, дом 23 E  реквизиты сведений об объекте контроля 45000000 инн по окопф форма собственности № п/п дата протокола по октмо 75203 29368873 45367000000 реквизиты документа, содержащего информацию для осуществления'

In [54]:
from nltk.tokenize import WordPunctTokenizer #Разделяем текст на слова токены

wpt = WordPunctTokenizer()

In [55]:
wpt.tokenize(data.text_part[0])

['771001001',
 'кпп',
 'по',
 'октмо',
 'гбу',
 '"',
 'жилищник',
 'района',
 'митино',
 '"',
 'государственные',
 'бюджетные',
 'учреждения',
 'субъектов',
 'российской',
 'федерации',
 '7733902394',
 'кпп',
 '773301001',
 'по',
 'окфс',
 'место',
 'нахождения',
 '(',
 'адрес',
 ')',
 'S',
 '125430',
 ',',
 'город',
 'москва',
 ',',
 'пятницкое',
 'шоссе',
 ',',
 'дом',
 '23',
 'E',
 'реквизиты',
 'сведений',
 'об',
 'объекте',
 'контроля',
 '45000000',
 'инн',
 'по',
 'окопф',
 'форма',
 'собственности',
 '№',
 'п',
 '/',
 'п',
 'дата',
 'протокола',
 'по',
 'октмо',
 '75203',
 '29368873',
 '45367000000',
 'реквизиты',
 'документа',
 ',',
 'содержащего',
 'информацию',
 'для',
 'осуществления']

In [56]:
def prepare_text(text):   # функция маркировки (маски mask) адреса начинается с S заканчивается E Например, в тексте:  S 125430, город москва, пятницкое шоссе, дом 23 E
    toks = wpt.tokenize(text)

    mask = []
    is_address = False

    for t in toks:
        if t == 'S':
            mask.append(0)
            is_address = True
            continue
        elif t == 'E':
            mask.append(0)
            is_address = False
            continue

        index = 1 if is_address else 0
        mask.append(index)

    text_df = pd.DataFrame({'token': toks, 'is_address': mask})    #Создаем Дата фрейм, где левый столбец это токен, а правый это маска, которая говорит этот токен часть адреса или нет
    text_df.loc[:, 'prev_info'] = text_df['is_address'].shift(1).fillna(0.0)

    text_df = text_df[~text_df['token'].isin(['S', 'E'])]

    hist = 3
    for i in range(1, hist+1):
        text_df.loc[:, f'hist_{i}'] = text_df['token'].shift(i)

    future = 3
    for i in range(1, future+1):
        text_df.loc[:, f'future{i}'] = text_df['token'].shift(-i)

    text_df = text_df.fillna('UNKNOWN')

    return text_df

In [9]:
text_dfs = []
for text in data.text_part.values:
    text_dfs.append(prepare_text(text))

<ipython-input-8-637db5583f44>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df.loc[:, f'hist_{i}'] = text_df['token'].shift(i)
<ipython-input-8-637db5583f44>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df.loc[:, f'hist_{i}'] = text_df['token'].shift(i)
<ipython-input-8-637db5583f44>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [10]:
len(text_dfs)

746

In [11]:
text_dfs = pd.concat(text_dfs)
text_dfs.head(5)

,token,is_address,prev_info,hist_1,hist_2,hist_3,future1,future2,future3
0,771001001,0,0.0,UNKNOWN,UNKNOWN,UNKNOWN,кпп,по,октмо
1,кпп,0,0.0,771001001,UNKNOWN,UNKNOWN,по,октмо,гбу
2,по,0,0.0,кпп,771001001,UNKNOWN,октмо,гбу,""""
3,октмо,0,0.0,по,кпп,771001001,гбу,"""",жилищник
4,гбу,0,0.0,октмо,по,кпп,"""",жилищник,района


In [12]:
text_dfs.shape

(74486, 9)

In [13]:
from sklearn.preprocessing import LabelEncoder  #Каждое слово, которое встретили будем отождествлять с каким от число, для подготовки данных к обработке дерревом решений

encoder = LabelEncoder()
encoder.fit(text_dfs['token'].tolist() + text_dfs['hist_1'].tolist())

LabelEncoder()

In [14]:
text_dfs.columns

Index(['token', 'is_address', 'prev_info', 'hist_1', 'hist_2', 'hist_3',
       'future1', 'future2', 'future3'],
      dtype='object')

In [15]:
digit_tdf = text_dfs.copy()

wrd_cols = ['token', 'hist_1', 'hist_2', 'hist_3', 'future1', 'future2', 'future3']
for wc in wrd_cols:
    digit_tdf.loc[:, wc] = encoder.transform(text_dfs[wc])

In [58]:
digit_tdf.head(5)

,token,is_address,prev_info,hist_1,hist_2,hist_3,future1,future2,future3
0,1373,0,0.0,1663,1663,1663,3408,4426,4120
1,3408,0,0.0,1373,1663,1663,4426,4120,2444
2,4426,0,0.0,3408,1373,1663,4120,2444,3
3,4120,0,0.0,4426,3408,1373,2444,3,2886
4,2444,0,0.0,4120,4426,3408,3,2886,4810


In [59]:
#Делим датасэт на признаки X и целевое значение y
X = digit_tdf.drop(['is_address'], axis=1)
y = digit_tdf['is_address']

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
X_train = X[:55000]
y_train = y[:55000]

X_test = X[55000:]
y_test = y[55000:]

In [19]:
model = RandomForestClassifier(max_depth=10)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=10)

In [29]:
X_test.loc[0, :]     #['ind']

,token,prev_info,hist_1,hist_2,hist_3,future1,future2,future3
0,5924,0.0,1663,1663,1663,4771,52,1912
0,103,0.0,1663,1663,1663,3,5307,65
0,225,0.0,1663,1663,1663,230,737,710
0,4969,0.0,1663,1663,1663,3721,4437,2927
0,3163,0.0,1663,1663,1663,4641,6053,1571
...,...,...,...,...,...,...,...,...
0,5497,0.0,1663,1663,1663,6053,4026,94
0,5460,0.0,1663,1663,1663,3594,4426,4417
0,5352,0.0,1663,1663,1663,603,74,231
0,3013,0.0,1663,1663,1663,5061,817,3067


In [34]:
ind

19484

In [31]:
#Делаем предсказание является ли  is_adress=1 для строчки, закодировавшей данный токен
X_test = X_test[:-1]
X_test.loc[:, 'prev_info'] = 0.0
X_test = X_test.reset_index(drop=True)

y_pred = []
for ind in X_test.index:
    pred = model.predict(X_test.loc[[ind]])[0]
    y_pred.append(pred)

    X_test.loc[ind+1, 'prev_info'] = pred

In [33]:
ind

19484

In [37]:
X_test.index

RangeIndex(start=0, stop=19486, step=1)

In [38]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.01      0.01     12906
           1       0.34      1.00      0.50      6580

    accuracy                           0.34     19486
   macro avg       0.52      0.50      0.26     19486
weighted avg       0.58      0.34      0.18     19486



In [39]:
y_test

,is_address
55,1
56,1
57,1
58,1
59,1
...,...
95,0
96,0
97,0
98,0


In [40]:
from sklearn.metrics import classification_report

# y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.01      0.01     12906
           1       0.34      1.00      0.50      6580

    accuracy                           0.34     19486
   macro avg       0.52      0.50      0.26     19486
weighted avg       0.58      0.34      0.18     19486



In [60]:
#Результат так себе. Применим словари, чтобы упростить решение

In [61]:
streets_df = pd.read_csv('/content/drive/MyDrive/Datasets/NLTK_neto/streets_df.csv') #выделим из каждого значения словаря с улицами Москвы слово (делаем у него строчный регистр)
streets_df = streets_df[['formalname']]
streets_df['formalname'] = streets_df['formalname'].str.lower()
streets_df.drop_duplicates(inplace=True)
streets_df.head(5)

,formalname
0,москва
1,новофедоровское
2,первомайское
3,филимонковское
4,киевский


In [43]:
streets_df.shape

(6493, 1)

In [44]:
streets_df = streets_df[streets_df['formalname'].str.len() > 3]

In [45]:
street_expr = '(' + '|'.join([s.replace('(', '\(').replace(')', '\)') for s in streets_df['formalname'].values]) + ')'
# street_expr

In [46]:
new_text_part = data.text_part.str.replace(street_expr, 'STREET')
new_text_part

,text_part
0,"771001001 кпп по октмо гбу ""жилищник района м..."
1,"5147746350230, окато 45283559000 октмо 453670..."
2,"он. статья 15 адреса, реквизиты и подписи стор..."
3,сударственное бюджетное учреждение гор. москвы...
4,остью «дорстройгрупп» (ооо «дорстройгрупп» юри...
...,...
741,тракту №обю/19-1 локальная смета № (локальный ...
742,ткая марка по плотности 70 длина 1000; 2000 мм...
743,"т 19.06.2017 № 508671, выданной публичным акци..."
744,и сжатии 300 или 400 400 вид цемента портландц...


In [47]:
new_text_part[0], data.text_part[0]

(' 771001001 кпп по октмо гбу "жилищник района митино" государственные бюджетные учреждения субъектов российской федерации 7733902394 кпп 773301001 по окфс место нахождения (адрес)  S 125430, город москва, пятницкое шоссе, дом 23 E  реквизиты сведений об объекте контроля 45000000 инн по окопф форма собственности № п/п дата протокола по октмо 75203 29368873 45367000000 реквизиты документа, содержащего информацию для осуществления',
 ' 771001001 кпп по октмо гбу "жилищник района митино" государственные бюджетные учреждения субъектов российской федерации 7733902394 кпп 773301001 по окфс место нахождения (адрес)  S 125430, город москва, пятницкое шоссе, дом 23 E  реквизиты сведений об объекте контроля 45000000 инн по окопф форма собственности № п/п дата протокола по октмо 75203 29368873 45367000000 реквизиты документа, содержащего информацию для осуществления')

In [48]:
text_dfs = []
for text in new_text_part:
    text_dfs.append(prepare_text(text))

<ipython-input-8-637db5583f44>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df.loc[:, f'hist_{i}'] = text_df['token'].shift(i)
<ipython-input-8-637db5583f44>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df.loc[:, f'hist_{i}'] = text_df['token'].shift(i)
<ipython-input-8-637db5583f44>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [49]:
text_dfs = pd.concat(text_dfs)
encoder = LabelEncoder()
encoder.fit(text_dfs['token'].tolist() + text_dfs['hist_1'].tolist())

digit_tdf = text_dfs.copy()

wrd_cols = ['token', 'hist_1', 'hist_2', 'hist_3', 'future1', 'future2', 'future3']
for wc in wrd_cols:
    digit_tdf.loc[:, wc] = encoder.transform(text_dfs[wc])

text_dfs.head(5)

,token,is_address,prev_info,hist_1,hist_2,hist_3,future1,future2,future3
0,771001001,0,0.0,UNKNOWN,UNKNOWN,UNKNOWN,кпп,по,октмо
1,кпп,0,0.0,771001001,UNKNOWN,UNKNOWN,по,октмо,гбу
2,по,0,0.0,кпп,771001001,UNKNOWN,октмо,гбу,""""
3,октмо,0,0.0,по,кпп,771001001,гбу,"""",жилищник
4,гбу,0,0.0,октмо,по,кпп,"""",жилищник,района


In [50]:
digit_tdf

,token,is_address,prev_info,hist_1,hist_2,hist_3,future1,future2,future3
0,1373,0,0.0,1663,1663,1663,3408,4426,4120
1,3408,0,0.0,1373,1663,1663,4426,4120,2444
2,4426,0,0.0,3408,1373,1663,4120,2444,3
3,4120,0,0.0,4426,3408,1373,2444,3,2886
4,2444,0,0.0,4120,4426,3408,3,2886,4810
...,...,...,...,...,...,...,...,...,...
95,38,0,0.0,2925,65,5396,3013,4144,5745
96,3013,0,0.0,38,2925,65,4144,5745,4328
97,4144,0,0.0,3013,38,2925,5745,4328,1663
98,5745,0,0.0,4144,3013,38,4328,1663,1663


In [51]:
X_train = X[:55000]
y_train = y[:55000]

X_test = X[55000:]
y_test = y[55000:]

model = RandomForestClassifier(max_depth=10)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=10)

In [52]:
X_test.loc[:, 'prev_info'] = 0.0
X_test = X_test.reset_index(drop=True)

y_pred = []
for ind in X_test.index:
    pred = model.predict(X_test.loc[[ind]])[0]
    y_pred.append(pred)

    X_test.loc[ind+1, 'prev_info'] = pred

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.01      0.01     12906
           1       0.34      1.00      0.50      6580

    accuracy                           0.34     19486
   macro avg       0.52      0.50      0.26     19486
weighted avg       0.58      0.34      0.18     19486

